In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../util')
sys.path.append('../experiments')

In [8]:
from convert_annotations import read_webanno, resolve_ellipses, webanno_to_iob_df, EMPTY_REGEX
from pathlib import Path
from datetime import datetime
import pandas as pd

In [12]:
ignore_documents = [
    '00_mundhoehlenkarzinom_0059.tsv',
    '00_mundhoehlenkarzinom_0071.tsv'
]

In [15]:
%%time
tsv_folder = '../../data_ggponc/output_ellipses/'
#tsv_folder = '../../ggponc_v2_annotation/other_layers/ellipses_dev'

webanno_df, sentences = read_webanno([f for f in Path(tsv_folder).glob('*.tsv') if not f.name in ignore_documents])

CPU times: user 10.7 s, sys: 322 ms, total: 11 s
Wall time: 11.8 s


In [16]:
%%time

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    iob_df = webanno_to_iob_df(webanno_df, 'detail', True, debug=False, collect_errors=False, skip_errors=True, all_columns=True)
    iob_df['id'] = iob_df.entity_id.where(~iob_df.entity_id.isna(), iob_df.spec_id)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49720/49720 [18:27<00:00, 44.90it/s]


CPU times: user 18min 36s, sys: 9.25 s, total: 18min 46s
Wall time: 18min 44s


In [17]:
# Anything containing a prefix, suffix or fragment is potentially an elliptical construct
ellipses = iob_df[~iob_df.prefix.str.match(EMPTY_REGEX) | ~iob_df.suffix.str.match(EMPTY_REGEX) | ~iob_df.fragment.str.match(EMPTY_REGEX)].copy()

In [54]:
ellipses.loc[, 12]

/tmp/ipykernel_1815141/2123648988.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  ellipses.loc['04_praevention-zervixkarzinom_0255.tsv', 12]


token_id  ts_id  \
file                                   sentence_id                    
04_praevention-zervixkarzinom_0255.tsv 12                 19  12-19   
                                       12                 20  12-20   
                                       12                 22  12-22   
                                       12                 23  12-23   

                                                         span   token  \
file                                   sentence_id                      
04_praevention-zervixkarzinom_0255.tsv 12           1379-1382     CIN   
                                       12           1383-1384       2   
                                       12           1385-1386       3   
                                       12           1387-1393  Läsion   

                                                          value  \
file                                   sentence_id                
04_praevention-zervixkarzinom_0255.tsv 12           Finding[18]   
                                       12           Finding[18]   
                                       12           Finding[19]   
                                       12           Finding[19]   

                                                                        detail  \
file                                   sentence_id                               
04_praevention-zervixkarzinom_0255.tsv 12           Diagnosis or Pathology[18]   
                                       12           Diagnosis or Pathology[18]   
                                       12           Diagnosis or Pathology[19]   
                                       12           Diagnosis or Pathology[19]   

                                                   specified_by     prefix  \
file                                   sentence_id                           
04_praevention-zervixkarzinom_0255.tsv 12                     _      *[18]   
                                       12                     _      *[18]   
                                       12                     _  CIN\_[19]   
                                       12                     _  CIN\_[19]   

                                                          suffix   fragment  \
file                                   sentence_id                            
04_praevention-zervixkarzinom_0255.tsv 12           \_Läsion[18]  12-22[19]   
                                       12           \_Läsion[18]  12-22[19]   
                                       12                  *[19]  12-19[18]   
                                       12                  *[19]  12-19[18]   

                                                    value_entity_id  \
file                                   sentence_id                    
04_praevention-zervixkarzinom_0255.tsv 12                      18.0   
                                       12                      18.0   
                                       12                      19.0   
                                       12                      19.0   

                                                   value_entity_class  \
file                                   sentence_id                      
04_praevention-zervixkarzinom_0255.tsv 12                     Finding   
                                       12                     Finding   
                                       12                     Finding   
                                       12                     Finding   

                                                   value_specification_id  \
file                                   sentence_id                          
04_praevention-zervixkarzinom_0255.tsv 12                             NaN   
                                       12                             NaN   
                                       12                             NaN   
                                       12                             NaN   

                 

In [19]:
%%time
results = resolve_ellipses(ellipses, iob_df)

../util/convert_annotations.py:166: PerformanceWarning: indexing past lexsort depth may impact performance.
  sentence = ellipses.loc[idx]
../util/convert_annotations.py:168: PerformanceWarning: indexing past lexsort depth may impact performance.
  full_sentence = iob_df.loc[idx]


CPU times: user 2min 11s, sys: 1.17 s, total: 2min 12s
Wall time: 2min 12s


### Adding resolved sentences and metadata

In [59]:
#idx = results[(results.file == '01_magenkarzinom_0090.tsv') & (results.sentence_id == 22)].index
#assert len(idx) == 1
#results.at[idx[0], 'resolution'] = ['Stadium', '2', 'und', 'Stadium 3']

In [60]:
sentence_idx = pd.DataFrame(sentences, columns=['raw_sentence'], index=webanno_df.index.drop_duplicates())

In [73]:
import math
clean_results = results.merge(right=sentence_idx, left_on=['file', 'sentence_id'], right_index=True)
clean_results['outer'] = math.nan

In [74]:
for _, k in clean_results[clean_results.duplicated(['file', 'sentence_id'])][['file', 'sentence_id']].iterrows():
    f = k['file']
    s = k['sentence_id']
    dups = clean_results[(clean_results.file == f) & (clean_results.sentence_id == s)]
    
    for i, inner in dups.iterrows():
        for j, outer in dups.iterrows():
            if i != j and inner.span_index_start >= outer.span_index_start and inner.span_index_end <= outer.span_index_end:
                if not (clean_results.loc[j, 'outer'] == i):
                    clean_results.loc[i, 'outer'] = j

In [75]:
clean_results[(clean_results.file == '04_praevention-zervixkarzinom_0255.tsv') & (clean_results.sentence_id == 12)]

,file,sentence_id,full_sentence,span_index_start,span_index_end,full_span,offsets,resolution,fragment,missing_prefix,missing_suffix,raw_sentence,outer
913,04_praevention-zervixkarzinom_0255.tsv,12,"[Ein, positiver, Absetzungsrand, zeigt, vergli...",18,22,"[CIN, 2, /, 3, Läsion]","[[116, 119], [120, 121], [121, 122], [122, 123...","[CIN, 2 Läsion, /, CIN 3, Läsion]",True,True,True,Ein positiver Absetzungsrand zeigt verglichen ...,914.0
914,04_praevention-zervixkarzinom_0255.tsv,12,"[Ein, positiver, Absetzungsrand, zeigt, vergli...",18,22,"[CIN, 2, /, 3, Läsion]","[[116, 119], [120, 121], [121, 122], [122, 123...","[CIN, 2 Läsion, /, CIN 3, Läsion]",True,True,True,Ein positiver Absetzungsrand zeigt verglichen ...,NaN


In [76]:
clean_results = clean_results[clean_results.outer.isna()]

In [77]:
def resolve(row):
    out = {}
    for c in row.index:
        val = row.loc[c]
        if len(val) == 1:
            out['multi'] = False
            out[c] = val[0]
        else:
            out['multi'] = True
            if val.count(val[0]) == len(val) and c in ['full_sentence', 'raw_sentence']:
                out[c] = val[0]
            elif c in ['fragment', 'missing_prefix', 'missing_suffix']:
                out[c] = max(val)
            else:
                out[c] = val
    return pd.Series(out)

In [78]:
clean_results = clean_results.groupby(['file', 'sentence_id', 'fragment']).aggregate(tuple).applymap(list).apply(resolve, axis=1)[list(clean_results.columns.drop(['outer', 'file', 'sentence_id', 'fragment'])) + ['multi']].reset_index()

In [79]:
def get_full_res(row):
    sent = row.raw_sentence
    if row.multi:
        spans = [o for p in row.offsets for o in p]
        resolution = [r for p in row.resolution for r in p]
    else:
        spans = row.offsets
        resolution = row.resolution
    assert len(spans) == len(resolution), (spans, resolution)
    j = 0
    for span, res in zip(spans, resolution):
        start, end = span
        start += j
        end += j
        sent = sent[:start] + res + sent[end:]
        j += (start - end + len(res))
    return sent

In [80]:
clean_results['full_resolution'] = clean_results.apply(get_full_res, axis=1)

In [81]:
import numpy as np

In [82]:
ellipses_sentence_count = len(results[['file', 'sentence_id']].drop_duplicates())
sentence_count = 78090
ellipses_sentence_count, sentence_count, ellipses_sentence_count / sentence_count

(4609, 78090, 0.059021641695479576)

In [83]:
print('Ellipses', len(results))
print('Forward ellipses', len(results[results.missing_prefix & ~results.missing_suffix]), len(results[results.fragment & results.missing_prefix & ~results.missing_suffix]))
print('Backward ellipses', len(results[~results.missing_prefix & results.missing_suffix]), len(results[results.fragment & ~results.missing_prefix & results.missing_suffix]))
print('Complex ellipses', len(results[results.missing_prefix & results.missing_suffix]), len(results[results.fragment & results.missing_prefix & results.missing_suffix]))

Ellipses 5187
Forward ellipses 1929 829
Backward ellipses 2955 2745
Complex ellipses 270 189


In [84]:
import random

def show_random(df):
    i = random.randint(0, len(df) - 1)
    item = df.iloc[i]    
    assert len(item.full_span) == len(item.resolution)
    spans = []
    solutions = []
    marker = []
    if not item.multi:
        for span, res in zip(item.full_span, item.resolution):
            if span == res:
                marker.append(' ' * len(span))
            else:
                marker.append('^' * len(res))
            start_index = res.find(span)
            spans.append(span.rjust(start_index + len(span)).ljust(len(res) - start_index))
            solutions.append(res)
        print(' '.join(spans))
        print(' '.join(solutions))
        print(' '.join(marker))
        print(f'{item.file};{item.sentence_id};{" ".join(item.full_span)}', 'fragment:', item.fragment)
    else:
         print(f'{item.file};{item.sentence_id}')
    print(item.raw_sentence)
    print(item.full_resolution)
    print('...........')
    print('')

In [85]:
for _ in range(0, 10):
    show_random(clean_results[clean_results.multi])

09_mammakarzinom_0644.tsv;3
Das erkrankungsfreie und Gesamtüberleben war signifikant höher in der CMF/AC-Gruppe (85% vs. 68% und 91% vs. 86%) verglichen mit der Capecitabinegruppe.
Das erkrankungsfreie Überleben und Gesamtüberleben war signifikant höher in der CMF-Gruppe/AC-Gruppe (85% vs. 68% und 91% vs. 86%) verglichen mit der Capecitabinegruppe.
...........

18_hodgkin-lymphom_0104.tsv;21
Die geplante Therapie bestand aus 2 Zyklen ABVD plus involved-field (IF-) Strahlentherapie für frühe, 4 Zyklen BEACOPP basis oder 4 Zyklen ABVD plus IF-Strahlentherapie für intermediäre und 6 bis 8 Zyklen BEACOPP basis gefolgt von einer lokalisierten Bestrahlung größerer Lymphomreste für fortgeschrittene Stadien.
Die geplante Therapie bestand aus 2 Zyklen ABVD plus involved-field (IF-) Strahlentherapie für frühe Stadien, 4 Zyklen BEACOPP basis oder 4 Zyklen ABVD plus IF-Strahlentherapie für intermediäre Stadien und 6 bis 8 Zyklen BEACOPP basis gefolgt von einer lokalisierten Bestrahlung größerer Ly

In [86]:
for _ in range(0, 10):
    show_random(clean_results[clean_results.fragment])

PALM             - , respektive COEIN-Komponenten
PALM-Komponenten  , respektive COEIN-Komponenten
^^^^^^^^^^^^^^^^                                
15_endometriumkarzinom_0042.tsv;10;PALM - , respektive COEIN-Komponenten fragment: True
Die allermeisten Ursachen der hier genannten PALM-, respektive COEIN-Komponenten können einer definierten Therapie zugeführt werden, dabei stehen nach dem Ausschluss operativ zu sanierender Befunde sowohl für die akute hämodynamisch nicht wirksame Blutung als auch für die chronische Blutungsstörung eine Reihe von medikamentösen Behandlungsverfahren zur Verfügung.
Die allermeisten Ursachen der hier genannten PALM-Komponenten, respektive COEIN-Komponenten können einer definierten Therapie zugeführt werden, dabei stehen nach dem Ausschluss operativ zu sanierender Befunde sowohl für die akute hämodynamisch nicht wirksame Blutung als auch für die chronische Blutungsstörung eine Reihe von medikamentösen Behandlungsverfahren zur Verfügung.
...........

tumor   

In [87]:
ss = clean_results[~clean_results.fragment][clean_results[~clean_results.fragment].resolution.map(lambda r: any([t for t in r if '-' in t]))]

In [88]:
for _ in range(0, 10):
    show_random(ss)

Untersuchung auf Mikrosatelliteninstabilität , NTRK-Fusionsgene , Amplifikationen von HER2 ,     die BRAF V600E Mutation oder Mutationen im IDH1 - Gen
Untersuchung auf Mikrosatelliteninstabilität , NTRK-Fusionsgene , Amplifikationen von HER2 , auf die BRAF V600E Mutation oder Mutationen im IDH1 - Gen
                                                                                             ^^^^^^^                                                  
11_hepatozellulaeres-karzinom-und-biliaere-karzinome_0260.tsv;13;Untersuchung auf Mikrosatelliteninstabilität , NTRK-Fusionsgene , Amplifikationen von HER2 , die BRAF V600E Mutation oder Mutationen im IDH1 - Gen fragment: False
Beispiele für molekulare Veränderungen, die therapeutisch genutzt werden können, sind neben FGFR2 insbesondere die Untersuchung auf Mikrosatelliteninstabilität, NTRK-Fusionsgene, Amplifikationen von HER2, die BRAF V600E Mutation oder Mutationen im IDH1-Gen.
Beispiele für molekulare Veränderungen, die therapeutisch gen

In [89]:
for _ in range(0, 10):
    show_random(clean_results[~clean_results.fragment])

gezielte psychoonkologische Unterstützung bei der Bewältigung der Erkrankung ,     der Behandlung und     der auftretenden Nebenwirkungen und Folgeprobleme
gezielte psychoonkologische Unterstützung bei der Bewältigung der Erkrankung , bei der Behandlung und bei der auftretenden Nebenwirkungen und Folgeprobleme
                                                                               ^^^^^^^                ^^^^^^^                                              
16_maligne-ovarialtumoren_0155.tsv;10;gezielte psychoonkologische Unterstützung bei der Bewältigung der Erkrankung , der Behandlung und der auftretenden Nebenwirkungen und Folgeprobleme fragment: False
Der behandelnde Arzt sollte über psychoonkologische Basiskompetenzen verfügen. Darüber hinaus sollte eine psychoonkologische Fachkraft in das Behandlungsteam integriert sein, um die psychoonkologische Beratung und Behandlung durchführen zu können. Die psychoonkologische Betreuung umfasst eine patientengerechte Information und Be

In [99]:
show_random(clean_results[(clean_results.file == '01_magenkarzinom_0033.tsv') & (clean_results.sentence_id == 4) & clean_results.fragment])

Beobachtungs        - und Kohortenstudien
Beobachtungsstudien  und Kohortenstudien
^^^^^^^^^^^^^^^^^^^                     
01_magenkarzinom_0033.tsv;4;Beobachtungs - und Kohortenstudien fragment: True
In den letzten Jahren wurde in einer Vielzahl von Beobachtungs- und Kohortenstudien der protektive Effekt von ASS, NSAR oder selektiven COX-2-Inhibitoren auf die Magenkarzinogenese untersucht [REF].
In den letzten Jahren wurde in einer Vielzahl von Beobachtungsstudien und Kohortenstudien der protektive Effekt von ASS, NSAR oder selektiven COX-2-Inhibitoren auf die Magenkarzinogenese untersucht [REF].
...........



In [93]:
date = datetime.now().strftime("%Y%m%d_%H%M%S")

In [94]:
date

'20220804_174013'

In [96]:
clean_results.to_excel(f'ellipses_nodup_{date}.xlsx')